### YOLO modeling

after testing the fealability of using pose estimators I now move to a newer model one that can support video anylsis better then the MPI heatmap-appraoch models

In [3]:
import torch
import numpy as np
import cv2
import ultralytics

print(f"✅ NumPy: {np.__version__}")       # Expect 1.26.x
print(f"✅ OpenCV: {cv2.__version__}")     # Expect 4.10.x
print(f"🚀 GPU Active: {torch.backends.mps.is_available()}")
print(ultralytics.__version__)                # Expect 8.2.0+

✅ NumPy: 1.26.4
✅ OpenCV: 4.10.0
🚀 GPU Active: False
8.3.241


In [7]:
# Helpers 
from typing import List, Union
import matplotlib.pyplot as plt
def show_image(img_or_list: Union[np.ndarray, List[np.ndarray]], row_plot: int = 1, titles: List[str] = None):
    """
    Display one or multiple images in a grid.
    - img_or_list: single array or list of arrays
    - row_plot: number of rows
    - titles: optional list of titles (same length as images)
    """
    imgs = img_or_list if isinstance(img_or_list, list) else [img_or_list]
    n = len(imgs)
    cols = int(np.ceil(n / row_plot))
    rows = row_plot

    fig, axes = plt.subplots(rows, cols, figsize=(4 * cols, 4 * rows))
    axes = np.array(axes).reshape(-1) if isinstance(axes, np.ndarray) else np.array([axes])

    for i, im in enumerate(imgs):
        if im.ndim == 2:
            axes[i].imshow(im, cmap='gray')
        else:
            axes[i].imshow(im)
        axes[i].axis('off')
        if titles is not None and i < len(titles):
            axes[i].set_title(titles[i])
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()



### Downloading the Model

In [ ]:
from ultralytics import YOLO # type: ignore

# 1. Load the model (YOLO11 is the 2025 standard)
model = YOLO('yolo11n-pose.pt') 

In [ ]:
import cv2



# . Open the input video
video_path = "./images/dancing.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# --- VIDEO WRITER SETUP ---
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_path = "./data_output/output_skeleton.mp4"

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

print(f"Processing video: {video_path}")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # 3. Run Pose Tracking
    # device='mps' uses the Mac GPU for faster processing
    results = model.track(
        frame,
        device='mps',
        persist=True,
        verbose=False,
        tracker='bytetrack.yaml' 
    )
    
    # 4. Draw the Skeleton
    # .plot() draws bounding boxes and the skeleton (keypoints + lines)
    annotated_frame = results[0].plot() 
    
    # 5. Write the frame to the output video file
    out.write(annotated_frame)

# 6. Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Done! The video with skeletons has been saved to: {output_path}")

Processing video: ./images/dancing.mp4
WARNING ⚠️ not enough matching points
WARNING ⚠️ not enough matching points
WARNING ⚠️ not enough matching points
